In [25]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")


def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)
tokenized_dataset = []
with open('./data/军事.txt', 'r') as f:
    for i, line in enumerate(f):
        temp = tokenize_function(line)
        temp['labels'] = i % 5
        tokenized_dataset.append(temp)


In [26]:
tokenized_dataset[1]

{'input_ids': [101, 1072, 3300, 749, 1408, 8043, 6716, 860, 5162, 6574, 1072, 1906, 749, 1408, 8043, 4289, 6574, 3175, 7481, 1377, 809, 2824, 1358, 2533, 749, 1658, 8043, 5341, 1394, 2131, 6225, 1469, 2544, 6225, 4384, 1862, 2798, 1962, 976, 1104, 2137, 8013, 976, 702, 6848, 2885, 4696, 4638, 1922, 7410, 8024, 6206, 5440, 5991, 1728, 3144, 1922, 1914, 1568, 8013, 8141, 2399, 2582, 720, 671, 2692, 2109, 6121, 6848, 2885, 3634, 1765, 8024, 2496, 3198, 1920, 3519, 3221, 862, 1728, 8013, 1744, 5093, 8024, 782, 4495, 817, 966, 6225, 8024, 679, 671, 3416, 5387, 749, 8024, 100, 113, 386, 142, 386, 114, 100, 1506, 1506, 172, 7178, 100, 2582, 720, 671, 4684, 6963, 5375, 8024, 784, 720, 3198, 952, 2798, 1377, 809, 679, 4500, 5440, 5991, 817, 3419, 1343, 3867, 6589, 8024, 7390, 2552, 2792, 3617, 1343, 976, 2682, 976, 4638, 691, 6205, 8013, 862, 3198, 2798, 5543, 3030, 5564, 4289, 6574, 677, 4638, 6577, 1737, 4307, 2578, 1450, 8013, 8013, 2769, 1346, 680, 749, 8024, 2832, 5314, 749, 107, 1092, 752

In [27]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_dataset, shuffle=False, batch_size=8)
for batch in train_dataloader:
    print(batch)
    break
# eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

{'input_ids': [tensor([101, 101, 101, 101, 101, 101, 101, 101]), tensor([ 1357,  1072,  3428, 10036,   702,   511,  6929,  1762]), tensor([5526, 3300,  679,  117,  704, 1259,  720, 1957]), tensor([  722,   749,  3171, 11685,  1744,  2886,  1377,  2961]), tensor([ 6887,  1408,  8024, 11160,  6568,   122,  4263,   686]), tensor([8026, 8043, 2135, 8765, 2168,  119,  117, 4518]), tensor([ 678, 6716, 1447, 9096, 1469,  119, 6929, 3344]), tensor([1286,  860, 5529, 9007, 5307,  119,  720, 5018]), tensor([ 2533,  5162,   744, 10291,  3845,  2945,  1004,   753]), tensor([4717, 6574, 1104,  106, 4638, 3308,  117, 6762]), tensor([ 833, 1072, 5032,  106, 1872, 7831, 1921, 3683]), tensor([1036, 1906, 1927, 4007, 7270,  704, 6070, 6612]), tensor([3219,  749, 2971, 1921,  511, 1925, 1392,  704]), tensor([3241, 1408, 8024, 3215,  100, 6858, 4905, 8024]), tensor([5125, 8043, 4384, 8013, 7988, 6380, 3300,  704]), tensor([4868, 4289,  924, 3123, 3175, 4852, 4263, 1744]), tensor([1168, 6574, 1927, 5487, 7

In [28]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [30]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [31]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [32]:
text = "I am yaoyuan's father"
tokens = tokenizer(text)
# print(tokens)
tokens = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in tokens.items()}

tokens

{'input_ids': tensor([[  101,   100,  8413, 12066, 12532,  9680,   112,   161, 12289, 10198,
            102]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [34]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for tokens in tokenized_dataset:
        tokens = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in tokens.items()}
        outputs = model(**tokens)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/3750 [00:00<?, ?it/s]

KeyboardInterrupt: 